In [345]:
import pandas as pd
import numpy as np
import json

In [346]:
data_dict = {}

In [347]:
_raw_data = pd.read_excel('AD_To_Fill_Event_Code_Configuration_v1.0.xlsx', header=list(range(4)), skipinitialspace=True, tupleize_cols=True, sheet_name = "6) Event code generater")

In [348]:
data = _raw_data.copy()

In [349]:
#Drop level from 
data.columns = pd.MultiIndex.from_tuples(data.columns).droplevel(2)

#Only keep frist 22 cols
data = data.iloc[:,:22]

In [350]:
data.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0               type  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
          Event_code         Event type           SIT Drop           Category   
0              10001              Audit        MVP1 Drop 1              Login   
1              10003              Audit        MVP1 Drop 1              Login   
2              10004              Audit        MVP1 Drop 1              Login   
3              10005              Audit        MVP1 Drop 1              Login   
4              10009              Audit        MVP1 Drop 3      Registration    

                           activityType                           category  \
  Unnamed: 4_level_1 Unnamed: 5_level_1                 Unnamed: 6_level_1   
           Epic Link     Activity type  Category (Financial/Non-financial)   
0              Login              Login                      Non-financial   
1              Login              Login                      Non-financial   
2              Login              Login                      Non-financial   
3              Login              Login                      Non-financial   
4       Registration       Registration                      Non-financial   

                      description  eventEngine                              \
               Unnamed: 7_level_1 targetTopics                               
                      Description    log.audit stream.documentum.documents   
0                       PIN Login            Y                         NaN   
1                 Biometric Login            Y                         NaN   
2        NEXT Term and Conditions            Y                         NaN   
3                          Logout            Y                         NaN   
4  Registration - ID verification            Y                         NaN   

   ...                           commsEngine                             \
   ...                          targetTopics                              
   ... stream.transaction.limit   push.email push.sms push.notification   
0  ...                      NaN            Y      NaN               NaN   
1  ...                      NaN            Y      NaN               NaN   
2  ...                      NaN          NaN      NaN                 Y   
3  ...                      NaN          NaN      NaN               NaN   
4  ...                      NaN          NaN      NaN               NaN   

                                                               \
  fromPreference                                  fromProfile   
      isRequired                    requiredField  isRequired   
0              Y  isLoginEmailNotificationEnabled           Y   
1              Y  isLoginEmailNotificationEnabled           Y   
2            NaN                              NaN           Y   
3            NaN                              NaN         NaN   
4            NaN                              NaN         NaN   

                             transactionAccumulator        
            pushNotification        transactionType        
  isImmediateRoutingRequired                    OWN OTHER  
0                        NaN                    NaN   NaN  
1                        NaN                    NaN   NaN  
2                        NaN                    NaN   NaN  
3                        NaN                    NaN   NaN  
4                        NaN                    NaN   NaN  

[5 rows x 22 columns]

#Replace Y with True values
data = data.replace(to_replace =["Y"],  value = True)

In [351]:
data.set_index(data.iloc[:,0], drop= True, inplace= True)

In [352]:
data.drop(labels= data.columns[0], axis= 'columns',inplace= True)

In [353]:
data.head()

Unnamed: 1_level_0  \
                                                   Unnamed: 1_level_1   
                                                           Event type   
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                      
10001                                                           Audit   
10003                                                           Audit   
10004                                                           Audit   
10005                                                           Audit   
10009                                                           Audit   

                                                   Unnamed: 2_level_0  \
                                                   Unnamed: 2_level_1   
                                                             SIT Drop   
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                      
10001                                                     MVP1 Drop 1   
10003                                                     MVP1 Drop 1   
10004                                                     MVP1 Drop 1   
10005                                                     MVP1 Drop 1   
10009                                                     MVP1 Drop 3   

                                                                 type  \
                                                   Unnamed: 3_level_1   
                                                             Category   
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                      
10001                                                           Login   
10003                                                           Login   
10004                                                           Login   
10005                                                           Login   
10009                                                   Registration    

                                                                       \
                                                   Unnamed: 4_level_1   
                                                            Epic Link   
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                      
10001                                                           Login   
10003                                                           Login   
10004                                                           Login   
10005                                                           Login   
10009                                                    Registration   

                                                         activityType  \
                                                   Unnamed: 5_level_1   
                                                       Activity type    
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                      
10001                                                           Login   
10003                                                           Login   
10004                                                           Login   
10005                                                           Login   
10009                                                    Registration   

                                                                             category  \
                                                                   Unnamed: 6_level_1   
                                                   Category (Financial/Non-financial)   
(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_...                                      
10001                                                                   Non-financial   
10003                                                                   Non-financial   
10004                                                                   Non-financial   
10005                                                                   Non-financial   
10009                                            

## Process description category activityType type

In [354]:
df_slice = data.iloc[:,[2,4,5,6]]
df_slice = df_slice.droplevel([1,2],axis=1)

In [355]:
data_dict  =  df_slice.to_dict('index')

## Process eventEngine

### Slice related columns

In [356]:
idx = pd.IndexSlice
df_slice = data.loc[:,idx['eventEngine','targetTopics']]
df_slice.head()

,log.audit,stream.documentum.documents,stream.ndid.evidence,stream.cbs-data.update,stream.transaction.limit
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,,,,
10001,Y,NaN,NaN,NaN,NaN
10003,Y,NaN,NaN,NaN,NaN
10004,Y,NaN,NaN,NaN,NaN
10005,Y,NaN,NaN,NaN,NaN
10009,Y,NaN,NaN,NaN,NaN


### Convert 'Y' to column name

In [357]:
df_slice = df_slice.loc[:, :].replace('Y', pd.Series(df_slice.columns, df_slice.columns))
df_slice.head()

,log.audit,stream.documentum.documents,stream.ndid.evidence,stream.cbs-data.update,stream.transaction.limit
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,,,,
10001,log.audit,NaN,NaN,NaN,NaN
10003,log.audit,NaN,NaN,NaN,NaN
10004,log.audit,NaN,NaN,NaN,NaN
10005,log.audit,NaN,NaN,NaN,NaN
10009,log.audit,NaN,NaN,NaN,NaN


### Create list of each row without NaN value

In [358]:
np_slice = df_slice.values
list_slice = [row[~pd.isnull(row)].tolist() for row in np_slice]

### Append list to dataframe

In [359]:
df_slice['targetTopics'] = list_slice

In [360]:
df_slice.head()

,log.audit,stream.documentum.documents,stream.ndid.evidence,stream.cbs-data.update,stream.transaction.limit,targetTopics
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,,,,,
10001,log.audit,NaN,NaN,NaN,NaN,[log.audit]
10003,log.audit,NaN,NaN,NaN,NaN,[log.audit]
10004,log.audit,NaN,NaN,NaN,NaN,[log.audit]
10005,log.audit,NaN,NaN,NaN,NaN,[log.audit]
10009,log.audit,NaN,NaN,NaN,NaN,[log.audit]


### Remove other columns

In [361]:
df_slice = df_slice.drop(df_slice.columns[0:5], axis=1)

In [362]:
df_slice

,targetTopics
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,[log.audit]
10003,[log.audit]
10004,[log.audit]
10005,[log.audit]
10009,[log.audit]
...,...
42019,[log.audit]
42020,[log.audit]
40067,"[log.audit, stream.cbs-data.update]"


### Remove rows with targetTopics = []

In [363]:
df_slice = df_slice[df_slice.astype(str)['targetTopics'] != '[]']

### Covert df to dict

In [364]:
df_slice_dict = df_slice.to_dict('index')

### Append with the existing dict

In [365]:
for key, value in data_dict.items():
    try:
        value['eventEngine'] = df_slice_dict[key]
    except:
        pass

## Process commsEngine

## targetTopics

In [366]:
df_slice = data.loc[:,idx['commsEngine','targetTopics']]
df_slice.head()

,push.email,push.sms,push.notification
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,,
10001,Y,NaN,NaN
10003,Y,NaN,NaN
10004,NaN,NaN,Y
10005,NaN,NaN,NaN
10009,NaN,NaN,NaN


In [367]:
df_slice = df_slice.loc[:, :].replace('Y', pd.Series(df_slice.columns, df_slice.columns))
df_slice.head()

,push.email,push.sms,push.notification
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,,
10001,push.email,NaN,NaN
10003,push.email,NaN,NaN
10004,NaN,NaN,push.notification
10005,NaN,NaN,NaN
10009,NaN,NaN,NaN


In [368]:
np_slice = df_slice.values
list_slice = [row[~pd.isnull(row)].tolist() for row in np_slice]

In [369]:
df_slice['targetTopics'] = list_slice

In [370]:
df_slice = df_slice.drop(df_slice.columns[0:3], axis=1)
df_slice.head()

,targetTopics
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,[push.email]
10003,[push.email]
10004,[push.notification]
10005,[]
10009,[]


In [371]:
df_slice = df_slice[df_slice.astype(str)['targetTopics'] != '[]']

In [372]:
targetTopics_dict = df_slice.to_dict('index')

In [373]:
targetTopics_dict

{10001: {'targetTopics': ['push.email']},
 10003: {'targetTopics': ['push.email']},
 10004: {'targetTopics': ['push.notification']},
 10010: {'targetTopics': ['push.email', 'push.sms']},
 21003: {'targetTopics': ['push.email']},
 21005: {'targetTopics': ['push.notification']},
 21013: {'targetTopics': ['push.email']},
 21014: {'targetTopics': ['push.email']},
 21016: {'targetTopics': ['push.notification']},
 21017: {'targetTopics': ['push.email']},
 21018: {'targetTopics': ['push.email']},
 21019: {'targetTopics': ['push.email']},
 21020: {'targetTopics': ['push.email']},
 21021: {'targetTopics': ['push.email']},
 21023: {'targetTopics': ['push.email']},
 21024: {'targetTopics': ['push.email']},
 21025: {'targetTopics': ['push.email']},
 21026: {'targetTopics': ['push.email']},
 21027: {'targetTopics': ['push.email']},
 21028: {'targetTopics': ['push.email']},
 21029: {'targetTopics': ['push.email']},
 21030: {'targetTopics': ['push.email']},
 21035: {'targetTopics': ['push.notificatio

## fromPreference

In [374]:
df_slice = data.loc[:,idx['commsEngine','fromPreference']]
df_slice.head()

,isRequired,requiredField
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,
10001,Y,isLoginEmailNotificationEnabled
10003,Y,isLoginEmailNotificationEnabled
10004,NaN,NaN
10005,NaN,NaN
10009,NaN,NaN


In [375]:
df_slice.loc[:, 'isRequired'] = df_slice.loc[:, 'isRequired'].replace('Y', True)
df_slice.head()

,isRequired,requiredField
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",,
10001,True,isLoginEmailNotificationEnabled
10003,True,isLoginEmailNotificationEnabled
10004,NaN,NaN
10005,NaN,NaN
10009,NaN,NaN


In [376]:
fromPreference_dict = df_slice.dropna('index',how='all').to_dict('index')

In [377]:
fromPreference_dict

{10001: {'isRequired': True,
  'requiredField': 'isLoginEmailNotificationEnabled'},
 10003: {'isRequired': True,
  'requiredField': 'isLoginEmailNotificationEnabled'},
 21003: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21013: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21014: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21015: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21017: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21018: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21019: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21020: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21021: {'isRequired': True,
  'requiredField': 'isTransactionEmailNotificationEnabled'},
 21023: {'isRequired':

## fromProfile

In [378]:
df_slice = data.loc[:,idx['commsEngine','fromProfile']]
df_slice.head()

,isRequired
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,Y
10003,Y
10004,Y
10005,NaN
10009,NaN


In [379]:
df_slice = df_slice.loc[:,:].replace('Y', True)
df_slice.head()

,isRequired
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,True
10003,True
10004,True
10005,NaN
10009,NaN


In [380]:
fromProfile_dict = df_slice.dropna('index',how='all').to_dict('index')

In [381]:
fromProfile_dict

{10001: {'isRequired': True},
 10003: {'isRequired': True},
 10004: {'isRequired': True},
 10010: {'isRequired': True},
 21003: {'isRequired': True},
 21005: {'isRequired': True},
 21013: {'isRequired': True},
 21014: {'isRequired': True},
 21015: {'isRequired': True},
 21016: {'isRequired': True},
 21017: {'isRequired': True},
 21018: {'isRequired': True},
 21019: {'isRequired': True},
 21020: {'isRequired': True},
 21021: {'isRequired': True},
 21023: {'isRequired': True},
 21024: {'isRequired': True},
 21025: {'isRequired': True},
 21026: {'isRequired': True},
 21027: {'isRequired': True},
 21028: {'isRequired': True},
 21029: {'isRequired': True},
 21030: {'isRequired': True},
 21035: {'isRequired': True},
 21036: {'isRequired': True},
 22000: {'isRequired': True},
 22002: {'isRequired': True},
 22003: {'isRequired': True},
 22004: {'isRequired': True},
 22015: {'isRequired': True},
 22016: {'isRequired': True},
 22017: {'isRequired': True},
 22018: {'isRequired': True},
 22019: {'

## pushNotification

In [382]:
df_slice = data.loc[:,idx['commsEngine','pushNotification']]
df_slice.head()

,isImmediateRoutingRequired
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,NaN
10003,NaN
10004,NaN
10005,NaN
10009,NaN


In [383]:
df_slice = df_slice.loc[:,:].replace('Y', True)
df_slice.head()

,isImmediateRoutingRequired
"(Unnamed: 0_level_0, Unnamed: 0_level_1, Event_code)",
10001,NaN
10003,NaN
10004,NaN
10005,NaN
10009,NaN


In [384]:
pushNotification_dict = df_slice.dropna('index',how='all').to_dict('index')

In [385]:
pushNotification_dict

{30060: {'isImmediateRoutingRequired': True},
 30071: {'isImmediateRoutingRequired': True},
 30072: {'isImmediateRoutingRequired': True},
 30073: {'isImmediateRoutingRequired': True},
 30074: {'isImmediateRoutingRequired': True}}

In [386]:
for key, value in data_dict.items():
    try:
        value['commsEngine'] = {
            'targetTopics': targetTopics_dict[key]
        }
    except:
        pass

In [387]:
for key, value in data_dict.items():
    try:
        value['commsEngine'] = {
            'fromPreference': fromPreference_dict[key]
        }
    except:
        pass

In [388]:
for key, value in data_dict.items():
    try:
        value['commsEngine'] = {
            'fromProfile': fromProfile_dict[key]
        }
    except:
        pass

In [389]:
for key, value in data_dict.items():
    try:
        value['commsEngine'] = {
            'pushNotification': pushNotification_dict[key]
        }
    except:
        pass

In [344]:
data_dict

: 'Statement ',
  'activityType': 'Account Loan ',
  'category': 'Non-financial',
  'description': 'Request e-Statement ',
  'eventEngine': {'targetTopics': ['log.audit']}},
 30104: {'type': 'Lock / Unlock',
  'activityType': 'ATM / Debit Card ',
  'category': 'Non-financial',
  'description': 'Lock Card',
  'eventEngine': {'targetTopics': ['log.audit']}},
 30105: {'type': 'Lock / Unlock',
  'activityType': 'ATM / Debit Card ',
  'category': 'Non-financial',
  'description': 'Unlock Card',
  'eventEngine': {'targetTopics': ['log.audit']}},
 30106: {'type': 'Verify by VISA ',
  'activityType': 'ATM / Debit Card ',
  'category': 'Non-financial',
  'description': 'Register Verify by VISA',
  'eventEngine': {'targetTopics': ['log.audit']}},
 40000: {'type': 'Account Settings',
  'activityType': 'Settings',
  'category': 'Non-financial',
  'description': 'Display Full Account Number ',
  'eventEngine': {'targetTopics': ['log.audit']}},
 40001: {'type': 'Transaction Limit',
  'activityType':

In [390]:
with open("test.json", 'w', encoding='utf-8') as f:
    json.dump(data_dict, f, ensure_ascii=False, indent=4)